## Импорт библиотек

In [0]:
%pip install -r ../requirements.txt

In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import sys
from glob import glob
from os.path import join
import os
import re
import shutil

np.random.seed(42)

## Парсинг телеграм чата

In [0]:
def get_str_from_file(path_to_file: str) -> str:
    '''Функция для полуения строки из текстового файла'''
    with open(path_to_file) as f:
        contents = f.readlines()
    # находим слившиеся со словами ссылки и разделям их
    text = ' '.join(content.strip().replace('http', ' http') if content.strip().find('http') not in [-1, 0] else content.strip() for content in contents)    
    # удаляем ссылки из сообщений
    text_without_links = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    return text_without_links

def read_html(path_to_html: str):
    '''Функция для чтения html'''
    with open(path_to_html, "r") as s:    
        contents = s.read() 
    soup = BeautifulSoup(contents, 'html.parser') 
    return soup

def save_msg_to_txt(soup, msg_id: str, output_path: str):
    '''Функция для сохранения сообщения в текстовый файл'''
    with open(output_path, 'w') as f:
        try:
            f.write(soup.body.find('div', id=msg_id).find_all('div', attrs={'class':"text"})[0].text)
        except IndexError:
            pass 

def create_df_from_dict(dictionary: dict, output_dir: str, df_name:str): 
    '''Функция преобразования словаря в DataFrame'''
    if len(dictionary) != 0:                         
        df = pd.DataFrame({
            'quest_id' : list(dictionary.keys()),
            'ans_id': list(dictionary.values())})
        df.to_csv(join(output_dir, df_name), index=False)

def parse_html(path_to_html: str):
    '''Функция для парсинга html файла'''
    
    # считаем html-документ
    soup = read_html(path_to_html)
    
    # словарь для хранения связей вопрос-ответ
    this_html_connect_dict = {}
     
    file_name = path_to_html.split('/')[-1].split('.')[0]
    connections_dir = join('..', 'data', 'connections_info')
    output_dir = join('..', 'data', 'connections_info', file_name)
    messages_dir = join('..', 'data', 'messages_text')
    
    # создаем необходимые папки
    os.makedirs(messages_dir, exist_ok=True)
    os.makedirs(connections_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)
    
    # получим id всех сообщений в html-документе
    msg_ids = [tag['id'] for tag in soup.select('div[id]')]
    
    with open(join(output_dir, file_name + '_info.txt'), 'w') as f:
        for elem in msg_ids:
            f.writelines(elem+'\n')
            
    for msg_id in msg_ids:
        # проверяем содержит ли msg_id в себе тег 'a' 
        a_list = soup.body.find('div', id=msg_id).find_all('a')
        if a_list != []:
            for a_elem in a_list:
                # если в тексте тега 'а' msg_id есть фраза 'this message', 
                # значит это сообщение является ответом на какое-то другое сообщение
                if a_elem.text == 'this message':
                    # получим id сообщения, на которое отвечали
                    question_id = 'message' + re.findall("\d+", a_elem.get('href'))[0]
                    # проверим есть ли сообщение, на которое отвечали (question_id), в текущем html-файле
                    if question_id in msg_ids:
                        quest_txt_path = join(messages_dir, question_id + '.txt')
                        ans_txt_path = join(messages_dir, msg_id + '.txt')
                        # создаем текстовый файл, только если его еще нет
                        if not os.path.exists(quest_txt_path):
                            save_msg_to_txt(soup=soup, msg_id=question_id, output_path=quest_txt_path)
                                 
                        if not os.path.exists(ans_txt_path):
                            save_msg_to_txt(soup=soup, msg_id=msg_id, output_path=ans_txt_path)
                            
                        if (os.path.exists(quest_txt_path) + os.path.exists(ans_txt_path)) == 2:
                            this_html_connect_dict[question_id] = msg_id

    # создаем df содержащий связи вопрос-ответ
    create_df_from_dict(dictionary=this_html_connect_dict,
                        output_dir=output_dir,
                        df_name='this_html_connect.csv')
    
def get_qa_txt_file(df_path: str, msg_dir: str, output_path: str):
    '''
    Функция создания текстового файла в формате:
    
    Q: ##текст вопроса##
    A: ##текст ответа##
    '''
    df = pd.read_csv(df_path)
    with open(output_path, "w") as outfile:
        for q, a in zip(df.quest_id.values,
                        df.ans_id.values):
            quest_path = join(msg_dir, q+'.txt')
            ans_path = join(msg_dir, a+'.txt')
            # проверяем существование txt файлов, а также наличие вопроса
            if ((os.path.exists(quest_path) + os.path.exists(ans_path)) == 2) and \
               (get_str_from_file(quest_path).find('?') != -1) and \
               (len(get_str_from_file(ans_path)) > 3):
                outfile.write('Q: ' + get_str_from_file(quest_path).strip() + '\n')
                outfile.write('A: ' + get_str_from_file(ans_path).strip() + '\n')

def create_dataset(list_of_txt: list, output_path: str):
    '''Функция создания датасета из нескольких текстовых файлов'''
    with open(output_path, 'w') as outfile:
        for fname in list_of_txt:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)

### 1. Выполняем парсинг тг чата

Полный сбор датасета выполняется достаточно долго, поэтому обработаем несколько файлов для сбора маленького датасета.

In [0]:
%%time
html_list = glob(join('..', 'data', 'ChatExport', '*.html'))
for html in html_list[:8]:
    parse_html(html)

### 2. Создаем общий текстовый файл с вопросами и ответами для каждого html-файла

In [0]:
%%time
messages_dir = join('..', 'data', 'messages_text')
connections_path = join('..', 'data', 'connections_info')
dir_list = [join(connections_path, directory) for directory in os.listdir(connections_path)]
for dir_path in dir_list:
    get_qa_txt_file(join(dir_path, 'this_html_connect.csv'), 
                    messages_dir,
                    join(dir_path, dir_path.split('/')[-1] + '.txt'))

### 3. Создаем датасет


In [0]:
%%time
os.makedirs(join('..', 'data', 'small_dataset'), exist_ok=True)
txt_list = [join(dir_path, dir_path.split('/')[-1] + '.txt') for dir_path in dir_list]
train_size = round(len(txt_list)*0.8)
create_dataset(list_of_txt=txt_list[:train_size], output_path=join('..', 'data', 'small_dataset','train.txt'))
create_dataset(list_of_txt=txt_list[train_size:], output_path=join('..', 'data', 'small_dataset','valid.txt'))

In [0]:
# Удалим лишние данные
shutil.rmtree('../data/ChatExport/')
shutil.rmtree('../data/messages_text/')